In [24]:
import numpy as np
import pandas as pd
from sklearn.ensemble import RandomForestClassifier
from sklearn.neighbors import KNeighborsClassifier
from sklearn.tree import DecisionTreeClassifier
from sklearn.preprocessing import PolynomialFeatures
from sklearn.model_selection import train_test_split
import numpy as np
from sklearn.model_selection import GridSearchCV
import matplotlib.pyplot as plt
from sklearn.ensemble import GradientBoostingRegressor
from sklearn.linear_model import LinearRegression, SGDRegressor
from sklearn.neural_network import MLPClassifier
from sklearn.linear_model import LogisticRegression
# import xgboost as xgb
from sklearn.metrics import accuracy_score
from sklearn.metrics import confusion_matrix
from sklearn.ensemble import RandomForestRegressor
from sklearn.model_selection import GridSearchCV
from sklearn.preprocessing import MinMaxScaler
from sklearn.metrics import classification_report, confusion_matrix
import pickle
from sklearn.naive_bayes import GaussianNB
# import json

# import websocket

In [25]:
 # Load data and set labels
data_positive = pd.read_csv('data/positive.csv', header=None)
column_labels = ['doc']
data_positive.columns = column_labels
data_positive['label'] = 1


# Display 5 random samples
data_positive.sample(5)

,doc,label
34884,"NOUS SOUSSIGNES, MEMBRES DU JURY DE L’EXAMEN ...",1
559,"NOUS SOUSSIGNES, MEMBRES DU JURY DE L’EXAMEN ...",1
4611,"NOUS SOUSSIGNES, MEMBRES DU JURY DE L’EXAMEN ...",1
10298,"NOUS SOUSSIGNES, MEMBRES DU JURY DE L’EXAMEN ...",1
18553,"NOUS SOUSSIGNES, MEMBRES DU JURY DE L’EXAMEN ...",1


In [26]:
data_negative1 = pd.read_csv('data/negative-1.csv')
data_negative1.columns = column_labels
data_negative1['label'] = 0

data_negative1.sample(5)

,doc,label
4993,"NOUS SOUSSIGNES, MEMBRES DU JURY DE L’EXAMEN ...",0
1172,"NOUS SOUSSIGNES, MEMBRES DU JURY DE L’EXAMEN ...",0
2056,"NOUS SOUSSIGNES, MEMBRES DU JURY DE L’EXAMEN ...",0
4682,"NOUS SOUSSIGNES, MEMBRES DU JURY DE L’EXAMEN ...",0
1375,"NOUS SOUSSIGNES, MEMBRES DU JURY DE L’EXAMEN ...",0


In [27]:
data_negative2 = pd.read_csv('data/negative-2.csv')
data_negative2.columns = column_labels
data_negative2['label'] = 0

data_negative3 = pd.read_csv('data/negative-3.csv')
data_negative3.columns = column_labels
data_negative3['label'] = 0
 

data_negative5 = pd.read_csv('data/negative-5.csv')
data_negative5.columns = column_labels
data_negative5['label'] = 0


data_negative6 = pd.read_csv('data/negative-6.csv')
data_negative6.columns = column_labels
data_negative6['label'] = 0

data_negative7 = pd.read_csv('data/negative-7.csv')
data_negative7.columns = column_labels
data_negative7['label'] = 0

data_negative8 = pd.read_csv('data/negative-8.csv')
data_negative8.columns = column_labels
data_negative8['label'] = 0

data_negative9 = pd.read_csv('data/negative-9.csv')
data_negative9.columns = column_labels
data_negative9['label'] = 0

data_negative9.sample(5)

,doc,label
1185,The abrupt ghost strikes poverty.The gamy lig...,0
696,The painstaking vet introduces industrializat...,0
2260,The productive mug juggles aquarium.The null ...,0
818,The aback portion pumps veldt.The petite cake...,0
2600,The exultant winner evaluates sponge.The holl...,0


In [28]:
# Concatenate complaining and non-complaining data
data = pd.concat([data_positive, data_negative1, data_negative2, data_negative3, data_negative5, data_negative6, data_negative7, data_negative8, data_negative9], axis=0).reset_index(drop=True)

In [29]:
# Display 5 random samples
data.sample(5)

,doc,label
34789,"NOUS SOUSSIGNES, MEMBRES DU JURY DE L’EXAMEN ...",1
20700,"NOUS SOUSSIGNES, MEMBRES DU JURY DE L’EXAMEN ...",1
3132,"NOUS SOUSSIGNES, MEMBRES DU JURY DE L’EXAMEN ...",1
63200,"NOUS SOUSSIGNES, MEMBRES DU JURY DE L’EXAMEN ...",0
313,"NOUS SOUSSIGNES, MEMBRES DU JURY DE L’EXAMEN ...",1


In [30]:
data.size

159984

In [31]:
data = data.sample(frac=1).reset_index(drop=True)

In [32]:
sampled_df = data.sample(frac=0.2) #let's take 32000 sample only for training

In [33]:
sampled_df.sample(3)

,doc,label
33188,"NOUS SOUSSIGNES, MEMBRES DU JURY DE L’EXAMEN ...",1
18375,PAR ORDONNANCE N° 88-092 DU 07 JUILLET 1998 ; ...,0
6611,"NOUS SOUSSIGNES, MEMBRES DU JURY DE L’EXAMEN ...",0


In [34]:
from sklearn.model_selection import train_test_split

X = sampled_df.doc.values
y = sampled_df.label.values

X_train, X_val, y_train, y_val =\
    train_test_split(X, y, test_size=0.2, random_state=2020)

In [35]:
import nltk
# Uncomment to download "stopwords"
nltk.download("stopwords")
from nltk.corpus import stopwords

def text_preprocessing(s):
    """
    - Lowercase the sentence
    - Change "'t" to "not"
    - Remove "@name"
    - Isolate and remove punctuations except "?"
    - Remove other special characters
    - Remove stop words except "not" and "can"
    - Remove trailing whitespace
    """
    s = s.lower()
    # Change 't to 'not'
    s = re.sub(r"\'t", " not", s)
    # Remove @name
    s = re.sub(r'(@.*?)[\s]', ' ', s)
    # Isolate and remove punctuations except '?'
    s = re.sub(r'([\'\"\.\(\)\!\?\\\/\,])', r'  ', s)
    s = re.sub(r'[^\w\s\?]', ' ', s)
    # Remove some special characters
    s = re.sub(r'([\;\:\|•«\n])', ' ', s)
    # Remove stopwords except 'not' and 'can'
    s = " ".join([word for word in s.split()
                  if word not in stopwords.words('english')
                  or word in ['not', 'can']])
    # Remove trailing whitespace
    s = re.sub(r'\s+', ' ', s).strip()

    return s

[nltk_data] Downloading package stopwords to C:\Users\YHLB2266\Données
[nltk_data]     d'applications\nltk_data...
[nltk_data]   Package stopwords is already up-to-date!


In [36]:
import re
from sklearn.feature_extraction.text import TfidfVectorizer

# Preprocess text
X_train_preprocessed = np.array([text_preprocessing(text) for text in X_train])
X_val_preprocessed = np.array([text_preprocessing(text) for text in X_val])

# Calculate TF-IDF
tf_idf = TfidfVectorizer(ngram_range=(1, 3),
                         binary=True,
                         smooth_idf=False)
X_train_tfidf = tf_idf.fit_transform(X_train_preprocessed)
X_val_tfidf = tf_idf.transform(X_val_preprocessed)

In [37]:
log_reg = LogisticRegression()
log_reg.fit(X_train_tfidf, y_train)

# Making predictions and evaluating the model
y_pred_log = log_reg.predict(X_val_tfidf)
print("Accuracy:", accuracy_score(y_val, y_pred_log)) 

Accuracy: 0.9028125


In [38]:
dtree = DecisionTreeClassifier()
dtree.fit(X_train_tfidf, y_train)

# Making predictions and evaluating the model
y_pred_dtree = dtree.predict(X_val_tfidf)
print("Accuracy:", accuracy_score(y_val, y_pred_dtree))  


Accuracy: 0.721875


In [39]:
rand_forest = RandomForestClassifier()
rand_forest.fit(X_train_tfidf, y_train)

# Making predictions and evaluating the model
y_pred_rand_forest = rand_forest.predict(X_val_tfidf)
print("Accuracy:", accuracy_score(y_val, y_pred_rand_forest)) 


Accuracy: 0.4653125


In [40]:
knn = KNeighborsClassifier()
knn.fit(X_train_tfidf, y_train)

# Making predictions and evaluating the model
y_pred_knn = knn.predict(X_val_tfidf)
print("Accuracy:", accuracy_score(y_val, y_pred_knn)) 


Accuracy: 0.6759375


In [42]:
from sklearn.svm import SVC

# Training the SVM model
svm_model = SVC(kernel='linear')  
svm_model.fit(X_train_tfidf, y_train)

# Making predictions and evaluating the model
y_pred_svm = svm_model.predict(X_val_tfidf)
print("Accuracy:", accuracy_score(y_val, y_pred_svm))
print("\nClassification Report:\n", classification_report(y_val, y_pred_svm))
print("\nConfusion Matrix:\n", confusion_matrix(y_val, y_pred_svm))

Accuracy: 0.9503125

Classification Report:
               precision    recall  f1-score   support

           0       1.00      0.90      0.95      1543
           1       0.91      1.00      0.95      1657

    accuracy                           0.95      3200
   macro avg       0.96      0.95      0.95      3200
weighted avg       0.95      0.95      0.95      3200


Confusion Matrix:
 [[1386  157]
 [   2 1655]]


In [ ]:
import seaborn as sns
# Confusion Matrix
conf_mat = confusion_matrix(y_val, y_pred_svm)
sns.heatmap(conf_mat, annot=True, fmt='d')
plt.xlabel('Predicted')
plt.ylabel('True')
plt.show()